In [1]:
# 구글 코랩 서버에 KoNLPy 라이브러리 설치
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 42.1 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
# 데이터 파일 읽어오기
# ratings_train.txt: 15만개의 영화평, 긍정과 부정이 각각 50%씩
# ratings_test.txt: 5만개의 영화평, 긍정과 부정이 각각 50%씩

def read_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    # txt 파일의 헤더 삭제하기
    data = data[1:]
  
  return data

train_data = read_data('/gdrive/My Drive/Colab Notebooks/movie/ratings_train.txt')
test_data = read_data('/gdrive/My Drive/Colab Notebooks/movie/ratings_test.txt')

print(len(train_data))
print(len(test_data))

150000
50000


In [4]:
# 훈련 데이터 5개 확인하기
for i in range(5):
  print(train_data[i])

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']
['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']
['10265843', '너무재밓었다그래서보는것을추천한다', '0']
['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0']
['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1']


In [5]:
# 테스트 데이터 5개 확인하기
for i in range(5):
  print(test_data[i])

['6270596', '굳 ㅋ', '1']
['9274899', 'GDNTOPCLASSINTHECLUB', '0']
['8544678', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0']
['6825595', '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0']
['6723715', '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0']


In [7]:
# 데이터 전처리 #1: KoNLPy 라이브러리를 이용하여 형태소 분석 및 품사 태깅

# 훈련 데이터와 시험 데이터에 대한 형태소 분석 및 품사 태깅
# 데이터 양이 큰 만큼 시간이 오래 걸리기 때문에 형태소 분석 및 품사 태깅 작업을 마친 후 json 파일로 저장
# 이미 작업이 완료된 train_docs.json과 test_docs.json 파일이 존재하면 분석 작업은 건너뜀

from konlpy.tag import Okt
import json
import os
import re

okt = Okt()

def tokenize(doc):
  # 특수 문자 제거
  doc = re.sub(r"[()!@#$%^&*]", "", doc)
  tmp = ['/' for t in okt.pos(doc, norm=True, stem=True)]
  ret_tokenize = []
  for token in tmp:
    (word, kind) = token.split('/')
    if kind != 'Josa':
      ret_tokenize.append(word)

  return ret_tokenize


with open('/gdrive/My Drive/Colab Notebooks/movie/train_docs2.json') as f:
  print('Train json file loading....')
  train_docs = json.load(f)
with open('/gdrive/My Drive/Colab Notebooks/movie/test_docs2.json') as f:
  print('Test json file loading....')
  test_docs = json.load(f)

Train json file loading....
Test json file loading....


In [8]:
# 토큰화한 결과 확인하기
for i in range(5):
  print(train_docs[i])

[['아', '더빙', '진짜', '짜증나다', '목소리'], '0']
[['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '않다'], '1']
[['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'], '0']
[['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], '0']
[['사이', '몬페', '그', '의', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '늙다', '보이다', '하다', '커스틴', '던스트', '너무나도', '이쁘다', '보이다'], '1']


In [9]:
# 데이터 전처리 #2: 데이터 벡터화
from tensorflow.keras.preprocessing.text import Tokenizer

train_x = [token for token, _ in train_docs]
test_x = [token for token, _ in test_docs]

print(train_x[0])
print(test_x[0])

['아', '더빙', '진짜', '짜증나다', '목소리']
['굳다', 'ㅋ']


In [10]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)
print(tokenizer.word_index) # 각 단어에 부여된 고유한 정수 확인

{'영화': 1, '하다': 2, '보다': 3, '이': 4, '들': 5, '없다': 6, '있다': 7, '좋다': 8, '너무': 9, '재밌다': 10, '정말': 11, '것': 12, '되다': 13, '적': 14, '같다': 15, '진짜': 16, '이다': 17, '점': 18, '아니다': 19, '않다': 20, '만들다': 21, '나오다': 22, '연기': 23, '평점': 24, '최고': 25, '내': 26, '그': 27, '안': 28, '왜': 29, '스토리': 30, '생각': 31, '못': 32, '드라마': 33, '다': 34, '감동': 35, '사람': 36, 'ㅋㅋㅋ': 37, '보고': 38, '1': 39, '말': 40, '아깝다': 41, '이렇다': 42, '더': 43, '때': 44, 'ㅋㅋ': 45, '배우': 46, '재미있다': 47, '감독': 48, '거': 49, '그냥': 50, '뭐': 51, '재미': 52, '내용': 53, '중': 54, '주다': 55, '한': 56, '재미없다': 57, '좀': 58, '자다': 59, '지루하다': 60, '쓰레기': 61, '수': 62, '시간': 63, '가다': 64, '모르다': 65, '들다': 66, '그렇다': 67, '싶다': 68, '10': 69, '작품': 70, '알다': 71, '사랑': 72, '나': 73, '^^': 74, '아': 75, '하나': 76, '다시': 77, '볼': 78, '마지막': 79, '잘': 80, 'ㅋ': 81, '저': 82, '이렇게': 83, '오다': 84, '처음': 85, '정도': 86, '완전': 87, '많다': 88, 'ㅠㅠ': 89, "'": 90, '장면': 91, '액션': 92, '주인공': 93, '2': 94, '안되다': 95, '개': 96, '차다': 97, '최악': 98, '하': 99, '나다': 100, '돈': 101, '이야기':

In [11]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [12]:
# 훈련 데이터 확인하기
for i in range(5):
  print(train_x[i])

[75, 435, 16, 223, 645]
[885, 436, 38, 590, 1, 200, 1422, 23, 659, 20]
[405, 2282, 5511, 3, 207, 34]
[6536, 102, 8736, 204, 52, 6, 24, 3580]
[1009, 27, 338, 9284, 23, 799, 1, 2551, 1087, 224, 2, 1050, 236, 224]


In [15]:
# 가장 긴 리뷰의 길이가 약 70정도이고, 리뷰의 평균 길이는 10정도 됨
# 전체 데이터의 길이는 30으로 맞춤

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 30

train_x = pad_sequences(train_x, maxlen= max_len)
test_x = pad_sequences(test_x, maxlen = max_len)

print(train_x[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  75 435  16 223 645]


In [16]:
for i in range(5):
  print(train_x[i])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  75 435  16 223 645]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  885  436   38  590    1  200 1422   23
  659   20]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  405 2282 5511    3
  207   34]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 6536  102 8736  204   52    6
   24 3580]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 1009   27  338 9284   23  799    1 2551 1087  224    2 1050
  236  224]


In [17]:
# y값 추출

train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [18]:
train_x[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  75,
       435,  16, 223, 645], dtype=int32)

In [19]:
train_y[0]

'0'

In [20]:
# 데이터 전처리 #3: 데이터를 numpy 배열에 저장하고, 데이터를 float로 형 변환
import numpy as np

x_train = train_x.astype('float32')
x_test = test_x.astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')



In [21]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((150000, 30), (50000, 30), (150000,), (50000,))

In [22]:
# 모델 정의
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,117,377
Trainable params: 1,117,377
Non-trainable params: 0
_________________________________________________________________


In [28]:
# 모델 학습과정 설정하기 & 학습시키기
 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=512)

Epoch 1/20
293/293 [==============================] - 8s 20ms/step - loss: 0.0671 - accuracy: 0.9734
Epoch 2/20
293/293 [==============================] - 6s 22ms/step - loss: 0.0602 - accuracy: 0.9762
Epoch 3/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0564 - accuracy: 0.9773
Epoch 4/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0529 - accuracy: 0.9789
Epoch 5/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0508 - accuracy: 0.9794
Epoch 6/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0494 - accuracy: 0.9803
Epoch 7/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0472 - accuracy: 0.9813
Epoch 8/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0441 - accuracy: 0.9823
Epoch 9/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0418 - accuracy: 0.9831
Epoch 10/20
293/293 [==============================] - 6s 20ms/step - loss: 0.0418 - accura

In [29]:
# 모델 평가하기
results = model.evaluate(x_test, y_test)
results

1563/1563 [==============================] - 9s 5ms/step - loss: 1.4957 - accuracy: 0.8279


[1.4957221746444702, 0.8278800249099731]

In [30]:
# 새로운 데이터로 결과 예측하기

def predict_pos_neg(review):
  token = [tokenize(review)]
  token = tokenizer.texts_to_sequences(token)
  token = pad_sequences(token, maxlen=max_len)
  token = token.astype('float32')
  score = model.predict(token)
  print(score)

predict_pos_neg('올해 최고의 영화! 세 번 넘게 봐도 질리지가 않아요')

[[0.42369813]]


In [31]:
predict_pos_neg('주연배우가 신인인데 연기 엄청 잘하네요.')
predict_pos_neg('너무 실망이에요. 돈이 아깝습니다.')

[[0.42369813]]
[[0.42369813]]
